참고 자료 : https://wingyu-story.tistory.com/4

네이버 github : https://github.com/naver/naver-openapi-guide/blob/master/sample/java/ApiExamShortenUrl.java

네이버 openapi : https://developers.naver.com/apps/#/myapps/E7oh3LML5WyT6UtsfCrA/overview


# **1. 개발 환경 설정**



## **1.1 필수 라이브러리 설치**

In [ ]:
!pip install --upgrade pip
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops gradio sentencepiece bitsandbytes


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb, platform, gradio, warnings
from datasets import load_dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login

In [ ]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd

In [ ]:
#허깅페이스 toekn을 넣으면 됩니다.
notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **2. 네이버 뉴스 크롤링**

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import json
import datetime
import urllib.parse
import re


# client_id = 'api-client-id 입력해주세요'
# client_secret = 'api-client-secret 입력해주세요'


# HTML 태그를 제거하는 함수
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header('X-Naver-Client-Id', client_id)
    req.add_header('X-Naver-Client-Secret', client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print('[%s] Url Request Success' % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print('[%s] Error for URL : %s' % (datetime.datetime.now(), url))
        return None

def getFullArticle(link):
    try:
        response = urllib.request.urlopen(link)
        if response.getcode() == 200:
            soup = BeautifulSoup(response, 'html.parser')
            article_body = soup.find('article', {'id': 'article-view-content-div'}).get_text()
            return article_body
        else:
            print(f'Failed to retrieve the article. Status code: {response.getcode()}')
            return None
    except Exception as e:
        return None

def getNaverSerach(node, srcText, start, display):
    base = 'https://openapi.naver.com/v1/search'
    node = '/%s.json' % node
    parameters = '?query=%s&start=%s&display=%s' % (urllib.parse.quote(srcText), start, display)


    url = base + node + parameters
    print(url)
    responseDecode = getRequestUrl(url)

    if responseDecode is None:
        return None
    else:
        search_results = json.loads(responseDecode)
        items = []
        for post in search_results['items']:
            print(post['originallink'])
            full_article = getFullArticle(post['originallink'])
            if full_article:
                post['fullArticle'] = full_article
                items.append(post)
        search_results['items'] = items
        return search_results


In [ ]:
results = getNaverSerach('news', '경제', 1, 10)
today_data = pd.DataFrame(results['items'])

# 'title' 열과 'description' 열에서 HTML 태그 제거
today_data['title'] = today_data['title'].apply(remove_html_tags)
today_data['description'] = today_data['description'].apply(remove_html_tags)
today_data

https://openapi.naver.com/v1/search/news.json?query=%EA%B2%BD%EC%A0%9C&start=1&display=10
[2024-06-12 03:30:40.311955] Url Request Success
http://www.betanews.net:8080/article/1468454.html
http://www.fnnews.com/news/202406121227436401
http://www.edaily.co.kr/news/newspath.asp?newsid=02371446638921064
https://www.ekn.kr/web/view.php?key=20240612024154804
https://www.farminsight.net/news/articleView.html?idxno=12771
http://www.veritas-a.com/news/articleView.html?idxno=508570
https://www.ekn.kr/web/view.php?key=20240612022314781
https://www.ekn.kr/web/view.php?key=20240612024564783
http://www.sisafocus.co.kr/news/articleView.html?idxno=319136
https://www.ekn.kr/web/view.php?key=20240612027404810


,title,originallink,link,description,pubDate,fullArticle
0,제주대 탐라문화연구원 '2024 제2회 쿰다콜로키움' 개최,http://www.veritas-a.com/news/articleView.html...,http://www.veritas-a.com/news/articleView.html...,"김성조 교수는 연세대 역사문화학과 교수로 한국현대사를 사회경제사, 문화사의 측면에서...","Wed, 12 Jun 2024 12:28:00 +0900",\n[베리타스알파=박소현 기자] 제주대 탐라문화연구원은 2024학년도 1학기 종강을...
1,'尹·與 겨냥' 이재명 &quot;놀려고 권력 위임···이 나라는 개인 사유물 아냐...,http://www.sisafocus.co.kr/news/articleView.ht...,http://www.sisafocus.co.kr/news/articleView.ht...,이 대표는 이날 국회 본청에서 열린 민주당 최고위원회의에서 &quot;며칠 사이 정...,"Wed, 12 Jun 2024 12:28:00 +0900",\n\n\n이재명 더불어민주당 대표가 지난 3일 서울 여의도 국회 본청에서 열린 최...


In [ ]:
today_data.to_csv('/content/drive/MyDrive/LLM fine tuning/naver_news.csv', index=False)

In [ ]:
today_data

,title,originallink,link,description,pubDate,fullArticle
0,제주대 탐라문화연구원 '2024 제2회 쿰다콜로키움' 개최,http://www.veritas-a.com/news/articleView.html...,http://www.veritas-a.com/news/articleView.html...,"김성조 교수는 연세대 역사문화학과 교수로 한국현대사를 사회경제사, 문화사의 측면에서...","Wed, 12 Jun 2024 12:28:00 +0900",\n[베리타스알파=박소현 기자] 제주대 탐라문화연구원은 2024학년도 1학기 종강을...
1,'尹·與 겨냥' 이재명 &quot;놀려고 권력 위임···이 나라는 개인 사유물 아냐...,http://www.sisafocus.co.kr/news/articleView.ht...,http://www.sisafocus.co.kr/news/articleView.ht...,이 대표는 이날 국회 본청에서 열린 민주당 최고위원회의에서 &quot;며칠 사이 정...,"Wed, 12 Jun 2024 12:28:00 +0900",\n\n\n이재명 더불어민주당 대표가 지난 3일 서울 여의도 국회 본청에서 열린 최...


# **3. 뉴스 기사 요약하기**

In [ ]:
today_data = pd.read_csv('/content/drive/MyDrive/LLM fine tuning/naver_news.csv')
today_data

,title,originallink,link,description,pubDate,fullArticle
0,제주대 탐라문화연구원 '2024 제2회 쿰다콜로키움' 개최,http://www.veritas-a.com/news/articleView.html...,http://www.veritas-a.com/news/articleView.html...,"김성조 교수는 연세대 역사문화학과 교수로 한국현대사를 사회경제사, 문화사의 측면에서...","Wed, 12 Jun 2024 12:28:00 +0900",\n[베리타스알파=박소현 기자] 제주대 탐라문화연구원은 2024학년도 1학기 종강을...
1,'尹·與 겨냥' 이재명 &quot;놀려고 권력 위임···이 나라는 개인 사유물 아냐...,http://www.sisafocus.co.kr/news/articleView.ht...,http://www.sisafocus.co.kr/news/articleView.ht...,이 대표는 이날 국회 본청에서 열린 민주당 최고위원회의에서 &quot;며칠 사이 정...,"Wed, 12 Jun 2024 12:28:00 +0900",\n\n\n이재명 더불어민주당 대표가 지난 3일 서울 여의도 국회 본청에서 열린 최...


In [ ]:
#모델 불러오기
#finetune_model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/LLM fine tuning/llama-2-7b-it-sum-ko_max_step=500_aihub", device_map='auto')#aihub데이터
finetune_model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/LLM fine tuning/llama-2-7b-it-sum-ko_max_step=500', device_map='auto')#네이버 뉴스 데이터
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', add_special_tokens=True)


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# 프롬프트 생성 함수
#title추가
def generate_prompt(title,doc):
    return f"""<bos><start_of_turn>user
다음 글을 요약해주세요:
{title}


{doc}<end_of_turn>
<start_of_turn>model"""

# 파이프라인 설정
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)


In [ ]:
def summarize_text(text):
  summaries = []
  for title,doc in zip(text['title'],text['fullArticle']):
    prompt = generate_prompt(title,doc)
    result = pipe_finetuned(prompt)
    summary = result[0]["generated_text"][len(prompt):].split('<end_of_turn>')[0].strip()
    summaries.append(summary)
  text['summary'] = summaries

  return text

In [ ]:
today_data=today_data
summar = summarize_text(today_data)
print('가져온 데이터 : %d 건' % (len(summar2)) )
summar


가져온 데이터 : 2 건


,title,originallink,link,description,pubDate,fullArticle,summary
0,제주대 탐라문화연구원 '2024 제2회 쿰다콜로키움' 개최,http://www.veritas-a.com/news/articleView.html...,http://www.veritas-a.com/news/articleView.html...,"김성조 교수는 연세대 역사문화학과 교수로 한국현대사를 사회경제사, 문화사의 측면에서...","Wed, 12 Jun 2024 12:28:00 +0900",\n[베리타스알파=박소현 기자] 제주대 탐라문화연구원은 2024학년도 1학기 종강을...,user\n\nuser\n\nuser\n\nuser\n\nuser\n\nuser\n...
1,'尹·與 겨냥' 이재명 &quot;놀려고 권력 위임···이 나라는 개인 사유물 아냐...,http://www.sisafocus.co.kr/news/articleView.ht...,http://www.sisafocus.co.kr/news/articleView.ht...,이 대표는 이날 국회 본청에서 열린 민주당 최고위원회의에서 &quot;며칠 사이 정...,"Wed, 12 Jun 2024 12:28:00 +0900",\n\n\n이재명 더불어민주당 대표가 지난 3일 서울 여의도 국회 본청에서 열린 최...,user\n\n다음 글을 요약해주세요:\n새 대통령 출마 희망 불확실 윤석열 대통령...


In [ ]:
summar.to_csv('/content/drive/MyDrive/LLM fine tuning/naver_news_summary.csv', index=False)

## **3.1 네이버뉴스 데이터셋으로 학습한 모델 요약결과**

In [ ]:
print(generate_prompt(summar['title'][0],summar['fullArticle'][0]))
print('-'*100)
print(summar['summary'][0])

<bos><start_of_turn>user
다음 글을 요약해주세요:
제주대 탐라문화연구원 '2024 제2회 쿰다콜로키움' 개최



[베리타스알파=박소현 기자] 제주대 탐라문화연구원은 2024학년도 1학기 종강을 맞이해 2024 두 번째 초청 강연 '쿰다콜로키움'을 13일 제주대 인문대학 1호관 대강당에서 개최한다고 12일 밝혔다. 강연자는 '새마을 운동'의 권위자인 김성조 교수이다.




   

김성조 교수는 연세대 역사문화학과 교수로 한국현대사를 사회경제사, 문화사의 측면에서 연구하고 있으며 근래에는 20세기 자본주의가 생활공간, 냉전문화, 생태환경, 몸(신체)을 어떻게 재구성했는지를 분석하고 있다. 특히, 역사학이 어떠한 새로운 서사와 세계인식을 가능케 할 수 있을지 검토하는 것에 관심을 갖고 있다. 연세대 글로벌한국학연구소 부소장, K학술확산연구소사업단 부단장, '역사비평' 편집위원 등을 역임했다. 
대표 논문으로는 The Political Economy of the Roof: The New Village Movement in 1970s South Korea (International Journal of Korean History 29-1, 2024); Land Reform and Postcolonial Poverty in South Korea, 1950-1970 (Agricultural History 95-2, 2021) ; 1970년대 농촌 주거공간의 변화와 소비자 농민: 인테리어 공간과 텔레비전 소비를 중심으로 (한국사연구 184, 2019) 등이 있다. 
강연 주제는 ''새마을' 공간의 정치경제학-한국현대사를 바라보는 몇 가지 새로운 서사'로, 새마을 운동에 대한 새로운 해석을 담고 있는 흥미로운 주제이다. 강연은 6월 13일 오후 2시에 제주대 인문대 1호관 대강당(8119호)에서 열릴 예정이다.
김치완 원장은 이번 강연이 "1970년대 한국사회의 발전과 변동에 관심이 있는 분들에게 많은 영감과 지식을 전해줄 귀한 자리가 될 것으로 기대하

In [ ]:
print(generate_prompt(summar['title'][1],summar['fullArticle'][1]))
print('-'*100)
print(summar['summary'][1])

<bos><start_of_turn>user
다음 글을 요약해주세요:
'尹·與 겨냥' 이재명 &quot;놀려고 권력 위임···이 나라는 개인 사유물 아냐&quot;





이재명 더불어민주당 대표가 지난 3일 서울 여의도 국회 본청에서 열린 최고위원회의에서 발언하고 있는 모습. 사진 / ⓒ뉴시스





 
[시사포커스 / 이혜영 기자] 이재명 더불어민주당 대표가 12일 해외 순방 중인 윤석열 대통령과 22대 국회 원 구성에 대한 문제를 제기하고 있는 집권 여당인 국민의힘을 향해 “권력을 줬더니 불필요한 생떼나 쓰고 보복이나 하고 있다”고 지적하면서 “이 나라를 개인 사유물로 여기는 것 같다”고 쏘아붙이며 작심 비판에 나섰다.
이 대표는 이날 국회 본청에서 열린 민주당 최고위원회의에서 “며칠 사이 정부·여당의 행태를 보면 국민은 죽든지 말든지, 나라 경제가 망쳐지든지 말든지 도대체 제대로 된 게 없다”며 “놀고 즐기자고 국민에게 권력을 위임받았느냐”고 따져 물으면서 이같이 밝혔다.
이어 그는 “권력 놀음하느라 즐거울지는 모르지만, 그 권력 뒤에 수없이 많은 사람이 가족들과 손잡고 죽느니 사느니 그러고 있다”며 “지금 이 순간에도 얼마나 많은 사람이 죽지 못해 살아가고 있는지를 생각해야 한다”고 목소리를 높였다.
그러면서 이 대표는 영세 자영업자의 대출 빚과 폐업율 상황을 언급하며 민생회복지원금에 대한 필요성을 거듭 강조하고 나섰는데, 이와 관련해 그는 “골목상권에 지역 화폐로 지급하면 돈이 없어지지 않고 돈다. 그러면 골목상권에 매출이 늘어나고 영세 자영업자들도 밀린 이자도 갚고 폐업도 미룰 수 있다”고 재차 설명했다.
아울러 이 대표는 국민의힘을 향해서도 자신들이 여당 몫으로 남겨둔 7개 상임위원장의 신속 구성을 촉구하는 모습을 에둘러 보여주기도 했는데, 실제로 그는 박찬대 원내대표를 향해 “7개 상임위를 여당에 구성해달라고 독촉하는 데 반응이 없다고 들었다”며 “그런데 언제까지 기다릴 것이냐. 법률상으론 지난 월요일(10일)에 했었어야 하는 것 아니냐”고

In [ ]:
print(generate_prompt(summar['title'][0],summar['fullArticle'][0]))
print('-'*100)
print(summar['summary'][0])

<bos><start_of_turn>user
다음 글을 요약해주세요:
[인사] 에너지경제연구원









[이넷뉴스] 
◇ 보직인사
 ○ 에너지정보통계센터 에너지조사통계연구실장 이재석 (연구위원)






 ○ 연구기획조정본부 연구기획팀장 김비아 (부연구위원)
[이넷뉴스=임효정 기자] im@enetnews.co.kr






 <end_of_turn>
<start_of_turn>model
----------------------------------------------------------------------------------------------------
이넷뉴스 인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보직인사 보�


In [ ]:
##기존 데이터셋
# #한국어 요약 모델 파인뉴팅 페이지 내의 데이터 셋으로 적용함.
from datasets import load_dataset
dataset_orgin = load_dataset("daekeun-ml/naver-news-summarization-ko")

In [ ]:
dataset_orgin['test']['document'][0]

'아이엘사이언스의 자회사 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 4일 밝혔다. 신규 특허는 자동차 주행 중 운전자의 졸음운전을 방지하는 상태 검출 기술에 관한 것이다. 해당 단말기는 가시광선 및 근적외선 광원을 조사하는 광원 모듈 운전자의 얼굴 영상을 촬영하는 가시광선 및 근적외선 카메라 차량 실내의 이산화탄소 농도를 측정하는 이산화탄소 센서로 구성됐다. 단말기는 광원에 반응하는 운전자의 얼굴 촬영 영상을 기반으로 심박 데이터와 눈의 깜빡임 횟수 눈을 감은 시간 등을 측정한다. 여기에 차내 졸음을 유발하는 이산화탄소 농도까지 종합적으로 분석해 운전자의 졸음 상태를 판단하고 결과값에 따라 경보 신호를 송출하도록 설계됐다. 아이트로닉스는 이번 특허기술을 차세대 지능형 교통체계 C ITS 시스템 설비에 적용할 예정이다. 회사 관계자는 이번 특허는 대표적인 차량 내적 사고 요인인 졸음운전 방지 차원에서 당사의 혁신 기술력을 집약해 정확도를 높이는 데 집중했다 며 완전 자율주행 단계에 이르기 전까지 지속될 운전자 안전사고 예방에 있어 해당 기술의 가시적인 성과를 기대하고 있다 고 말했다.'

In [ ]:
prompt = generate_prompt(dataset_orgin['test']['title'][0],dataset_orgin['test']['document'][0])
print(prompt)
result = pipe_finetuned(prompt)

<bos><start_of_turn>user
다음 글을 요약해주세요:
아이트로닉스 차량용 복합기능형 졸음 방지 단말기 특허 출원


아이엘사이언스의 자회사 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 4일 밝혔다. 신규 특허는 자동차 주행 중 운전자의 졸음운전을 방지하는 상태 검출 기술에 관한 것이다. 해당 단말기는 가시광선 및 근적외선 광원을 조사하는 광원 모듈 운전자의 얼굴 영상을 촬영하는 가시광선 및 근적외선 카메라 차량 실내의 이산화탄소 농도를 측정하는 이산화탄소 센서로 구성됐다. 단말기는 광원에 반응하는 운전자의 얼굴 촬영 영상을 기반으로 심박 데이터와 눈의 깜빡임 횟수 눈을 감은 시간 등을 측정한다. 여기에 차내 졸음을 유발하는 이산화탄소 농도까지 종합적으로 분석해 운전자의 졸음 상태를 판단하고 결과값에 따라 경보 신호를 송출하도록 설계됐다. 아이트로닉스는 이번 특허기술을 차세대 지능형 교통체계 C ITS 시스템 설비에 적용할 예정이다. 회사 관계자는 이번 특허는 대표적인 차량 내적 사고 요인인 졸음운전 방지 차원에서 당사의 혁신 기술력을 집약해 정확도를 높이는 데 집중했다 며 완전 자율주행 단계에 이르기 전까지 지속될 운전자 안전사고 예방에 있어 해당 기술의 가시적인 성과를 기대하고 있다 고 말했다.<end_of_turn>
<start_of_turn>model


In [ ]:
result[0]["generated_text"][len(prompt):]

'\n4일 4일 아이엘사이언스의 자회사 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 밝혔는데 해당 특허는 자동차 주행 중 운전자의 졸음운전을 방지하는 상태 검출 기술에 관한 것이며 해당 단말기는 가시광선 및 근적외선 광원을 조사하는 광원 모듈 운전자의 얼굴 영상을 촬영하는 가시광선 및 근적외선 카메라 차량 실내의 이산화탄소 농도를 측정하는 이산화탄소 센서로 구성됐는데 단말기는 광원에 반응하는 운전자의 얼굴 촬영 영상을 기반으로 심박 데이터와 눈의 깜빡임 횟수 눈을 감은 시간 등을 측정한다. 여기에 차내 졸음을 유발하는 이산화탄소 농도까지 종합적으로 분석해 운'

## **3.2 AI-Hub 데이터셋으로 학습한 모델 요약결과**

In [ ]:
print(generate_prompt(summar2['title'][0],summar2['fullArticle'][0]))
print('-'*100)
print(summar2['summary'][0])

<bos><start_of_turn>user
다음 글을 요약해주세요:
제주대 탐라문화연구원 '2024 제2회 쿰다콜로키움' 개최



[베리타스알파=박소현 기자] 제주대 탐라문화연구원은 2024학년도 1학기 종강을 맞이해 2024 두 번째 초청 강연 '쿰다콜로키움'을 13일 제주대 인문대학 1호관 대강당에서 개최한다고 12일 밝혔다. 강연자는 '새마을 운동'의 권위자인 김성조 교수이다.




   

김성조 교수는 연세대 역사문화학과 교수로 한국현대사를 사회경제사, 문화사의 측면에서 연구하고 있으며 근래에는 20세기 자본주의가 생활공간, 냉전문화, 생태환경, 몸(신체)을 어떻게 재구성했는지를 분석하고 있다. 특히, 역사학이 어떠한 새로운 서사와 세계인식을 가능케 할 수 있을지 검토하는 것에 관심을 갖고 있다. 연세대 글로벌한국학연구소 부소장, K학술확산연구소사업단 부단장, '역사비평' 편집위원 등을 역임했다. 
대표 논문으로는 The Political Economy of the Roof: The New Village Movement in 1970s South Korea (International Journal of Korean History 29-1, 2024); Land Reform and Postcolonial Poverty in South Korea, 1950-1970 (Agricultural History 95-2, 2021) ; 1970년대 농촌 주거공간의 변화와 소비자 농민: 인테리어 공간과 텔레비전 소비를 중심으로 (한국사연구 184, 2019) 등이 있다. 
강연 주제는 ''새마을' 공간의 정치경제학-한국현대사를 바라보는 몇 가지 새로운 서사'로, 새마을 운동에 대한 새로운 해석을 담고 있는 흥미로운 주제이다. 강연은 6월 13일 오후 2시에 제주대 인문대 1호관 대강당(8119호)에서 열릴 예정이다.
김치완 원장은 이번 강연이 "1970년대 한국사회의 발전과 변동에 관심이 있는 분들에게 많은 영감과 지식을 전해줄 귀한 자리가 될 것으로 기대하

In [ ]:
print(generate_prompt(summar2['title'][1],summar2['fullArticle'][1]))
print('-'*100)
print(summar2['summary'][1])

<bos><start_of_turn>user
다음 글을 요약해주세요:
'尹·與 겨냥' 이재명 &quot;놀려고 권력 위임···이 나라는 개인 사유물 아냐&quot;





이재명 더불어민주당 대표가 지난 3일 서울 여의도 국회 본청에서 열린 최고위원회의에서 발언하고 있는 모습. 사진 / ⓒ뉴시스





 
[시사포커스 / 이혜영 기자] 이재명 더불어민주당 대표가 12일 해외 순방 중인 윤석열 대통령과 22대 국회 원 구성에 대한 문제를 제기하고 있는 집권 여당인 국민의힘을 향해 “권력을 줬더니 불필요한 생떼나 쓰고 보복이나 하고 있다”고 지적하면서 “이 나라를 개인 사유물로 여기는 것 같다”고 쏘아붙이며 작심 비판에 나섰다.
이 대표는 이날 국회 본청에서 열린 민주당 최고위원회의에서 “며칠 사이 정부·여당의 행태를 보면 국민은 죽든지 말든지, 나라 경제가 망쳐지든지 말든지 도대체 제대로 된 게 없다”며 “놀고 즐기자고 국민에게 권력을 위임받았느냐”고 따져 물으면서 이같이 밝혔다.
이어 그는 “권력 놀음하느라 즐거울지는 모르지만, 그 권력 뒤에 수없이 많은 사람이 가족들과 손잡고 죽느니 사느니 그러고 있다”며 “지금 이 순간에도 얼마나 많은 사람이 죽지 못해 살아가고 있는지를 생각해야 한다”고 목소리를 높였다.
그러면서 이 대표는 영세 자영업자의 대출 빚과 폐업율 상황을 언급하며 민생회복지원금에 대한 필요성을 거듭 강조하고 나섰는데, 이와 관련해 그는 “골목상권에 지역 화폐로 지급하면 돈이 없어지지 않고 돈다. 그러면 골목상권에 매출이 늘어나고 영세 자영업자들도 밀린 이자도 갚고 폐업도 미룰 수 있다”고 재차 설명했다.
아울러 이 대표는 국민의힘을 향해서도 자신들이 여당 몫으로 남겨둔 7개 상임위원장의 신속 구성을 촉구하는 모습을 에둘러 보여주기도 했는데, 실제로 그는 박찬대 원내대표를 향해 “7개 상임위를 여당에 구성해달라고 독촉하는 데 반응이 없다고 들었다”며 “그런데 언제까지 기다릴 것이냐. 법률상으론 지난 월요일(10일)에 했었어야 하는 것 아니냐”고

In [ ]:
summar2.to_csv('/content/drive/MyDrive/LLM fine tuning/naver_news_summary2.csv', index=False)

In [ ]:
##기존 데이터셋
# #한국어 요약 모델 파인뉴팅 페이지 내의 데이터 셋으로 적용함.
from datasets import load_dataset
dataset_orgin = load_dataset("daekeun-ml/naver-news-summarization-ko")

In [ ]:
# 프롬프트 생성 함수
#title추가
def generate_prompt(title,doc):
    return f"""<bos><start_of_turn>user
다음 글을 요약해주세요:

{doc}<end_of_turn>
<start_of_turn>model"""

# 파이프라인 설정
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)


In [ ]:
dataset_orgin['test']['document'][0]

'아이엘사이언스의 자회사 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 4일 밝혔다. 신규 특허는 자동차 주행 중 운전자의 졸음운전을 방지하는 상태 검출 기술에 관한 것이다. 해당 단말기는 가시광선 및 근적외선 광원을 조사하는 광원 모듈 운전자의 얼굴 영상을 촬영하는 가시광선 및 근적외선 카메라 차량 실내의 이산화탄소 농도를 측정하는 이산화탄소 센서로 구성됐다. 단말기는 광원에 반응하는 운전자의 얼굴 촬영 영상을 기반으로 심박 데이터와 눈의 깜빡임 횟수 눈을 감은 시간 등을 측정한다. 여기에 차내 졸음을 유발하는 이산화탄소 농도까지 종합적으로 분석해 운전자의 졸음 상태를 판단하고 결과값에 따라 경보 신호를 송출하도록 설계됐다. 아이트로닉스는 이번 특허기술을 차세대 지능형 교통체계 C ITS 시스템 설비에 적용할 예정이다. 회사 관계자는 이번 특허는 대표적인 차량 내적 사고 요인인 졸음운전 방지 차원에서 당사의 혁신 기술력을 집약해 정확도를 높이는 데 집중했다 며 완전 자율주행 단계에 이르기 전까지 지속될 운전자 안전사고 예방에 있어 해당 기술의 가시적인 성과를 기대하고 있다 고 말했다.'

In [ ]:
prompt = generate_prompt(dataset_orgin['test']['title'][0],dataset_orgin['test']['document'][0])
print(prompt)
result = pipe_finetuned(prompt)

<bos><start_of_turn>user
다음 글을 요약해주세요:
아이트로닉스 차량용 복합기능형 졸음 방지 단말기 특허 출원


아이엘사이언스의 자회사 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 4일 밝혔다. 신규 특허는 자동차 주행 중 운전자의 졸음운전을 방지하는 상태 검출 기술에 관한 것이다. 해당 단말기는 가시광선 및 근적외선 광원을 조사하는 광원 모듈 운전자의 얼굴 영상을 촬영하는 가시광선 및 근적외선 카메라 차량 실내의 이산화탄소 농도를 측정하는 이산화탄소 센서로 구성됐다. 단말기는 광원에 반응하는 운전자의 얼굴 촬영 영상을 기반으로 심박 데이터와 눈의 깜빡임 횟수 눈을 감은 시간 등을 측정한다. 여기에 차내 졸음을 유발하는 이산화탄소 농도까지 종합적으로 분석해 운전자의 졸음 상태를 판단하고 결과값에 따라 경보 신호를 송출하도록 설계됐다. 아이트로닉스는 이번 특허기술을 차세대 지능형 교통체계 C ITS 시스템 설비에 적용할 예정이다. 회사 관계자는 이번 특허는 대표적인 차량 내적 사고 요인인 졸음운전 방지 차원에서 당사의 혁신 기술력을 집약해 정확도를 높이는 데 집중했다 며 완전 자율주행 단계에 이르기 전까지 지속될 운전자 안전사고 예방에 있어 해당 기술의 가시적인 성과를 기대하고 있다 고 말했다.<end_of_turn>
<start_of_turn>model


In [ ]:
result[0]["generated_text"][len(prompt):]

'\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nuser\nu